In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [2]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [3]:

from pandas.io import sql as psql

In [4]:
from lib.connected_db import PgsqlExecutor

In [5]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [6]:
db = PgsqlExecutor(snpsource)
model_db = PgsqlExecutor(research_db)

# Long investment - consistent growth up

In [7]:
def get_long_stock_data(year):
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    and is_yearly_consistent_rate = 1
    and average_growth_rate > 0
    order by adjusted_growth desc
    limit 30

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data

def get_short_stock_data(year):
    """
    You don't want to short pennies stock, you only care when stock > 10
    """
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    and is_yearly_consistent_rate = 1
    and average_growth_rate < 0
    and price > 10
    order by adjusted_growth
    limit 30

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data
    


In [8]:
REPORT = """
**********- Score: {score} - {symbol} - {exchange}: {start_price} on {start_date} - *********
End date: {end_date} - price: {end_price} - min: {min_price} - max: {max_price}
Profit: {perf_profit} - max potential: {perf_potential} 
Average growth rate: {avg_growth}, consistent: {is_consistent}, project growth: {adj_growth}
DCF: {dcf}, market_dcf: {market_dcf}, adjusted_dcf: {adj_dcf}, dcf_to_price : {dcf_to_price}

"""
import numpy

def report_all(year):
    profits = []
    potentials = []
    data = get_long_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.year_end_price - row.price) * 100 / row.price
            ,1)
        potential = round(
            (row.year_max_price - row.price) * 100 / row.price
            ,1)
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% ***".format(
        numpy.mean(profits), numpy.mean(potentials)
    ))
    return numpy.mean(profits), numpy.mean(potentials)

def report_short_all(year):
    profits = []
    potentials = []
    data = get_short_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.price - row.year_end_price) * 100 / row.price
            ,1)
        potential = round(
            (row.price - row.year_min_price) * 100 / row.price
            ,1)
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% ***".format(
        numpy.mean(profits), numpy.mean(potentials)
    ))
    return numpy.mean(profits), numpy.mean(potentials)


# Including Pennies Stock - 2019

In [9]:
report_all(2019)


**********- Score: -9.399999618530273 - MCY - NYSE: 48.73 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 41.93 - min: 34.15 - max: 53.7
Profit: -14.0 - max potential: 10.2 
Average growth rate: 0.796, consistent: 1, project growth: 0.519
DCF: 318.8, market_dcf: 77.3, adjusted_dcf: 170.2, dcf_to_price : 249.27



**********- Score: 6.400000095367432 - EXEL - NasdaqGS: 17.62 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 22.47 - min: 14.46 - max: 27.42
Profit: 27.5 - max potential: 55.6 
Average growth rate: 0.762, consistent: 1, project growth: 0.517
DCF: 24.0, market_dcf: 16.1, adjusted_dcf: 19.6, dcf_to_price : 11.24



**********- Score: -13.899999618530273 - QTWO - NYSE: 81.08 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 81.31 - min: 50.63 - max: 89.83
Profit: 0.3 - max potential: 10.8 
Average growth rate: 0.735, consistent: 1, project growth: 0.477
DCF: 87.4, market_dcf: 78.7, adjusted_dcf: 82.2, d

(6.103333333333336, 47.75666666666667)

# 2018

In [31]:
report_all(2018)


**********- Score: 5.800000190734863 - AGO - NYSE: 38.28 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 49.02 - min: 37.88 - max: 50.51
Profit: 28.1 - max potential: 31.9 
Average growth rate: 10.92, consistent: 1, project growth: 7.088
DCF: 927398.2, market_dcf: 55.9, adjusted_dcf: 131649.7, dcf_to_price : 343812.49



**********- Score: 4.699999809265137 - VVUS - NasdaqGS: 2.23 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 2.72 - min: 2.23 - max: 5.48
Profit: 22.0 - max potential: 145.7 
Average growth rate: 7.533, consistent: 1, project growth: 5.184
DCF: -16902950.7, market_dcf: -584.2, adjusted_dcf: -3379366.6, dcf_to_price : -151541203.14



**********- Score: 23.899999618530273 - LCTC - OTCPK: 5.15 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 4.8 - min: 0.0 - max: 6.25
Profit: -6.8 - max potential: 21.4 
Average growth rate: 5.589, consistent: 1, project growth: 3.608
DCF: -8418.9, market_dcf: 4

(19.1, 66.38333333333334)

In [32]:
initial_investment = 1000
potential_gain = initial_investment
yearly_cumulated_gain = initial_investment
potential_yearly_gains = []
profit_yearly_gains = []

for year in range(2005, 2010):
    print (year)
    profit_per, potential_per = report_all(year)
    profit_yearly_gains.append(profit_per)
    potential_yearly_gains.append(potential_per)
    
    yearly_cumulated_gain = yearly_cumulated_gain * (1 + profit_per/100)
    potential_gain = potential_gain *(1 + potential_per/100)



2005

**********- Score: 3.700000047683716 - BIIB - NasdaqGS: 45.28 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 49.19 - min: 40.73 - max: 52.26
Profit: 8.6 - max potential: 15.4 
Average growth rate: 10.577, consistent: 1, project growth: 7.372
DCF: 308114.6, market_dcf: 44.3, adjusted_dcf: 61736.1, dcf_to_price : 136242.98



**********- Score: 7.300000190734863 - USLM - NasdaqGS: 26.47 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 30.15 - min: 23.16 - max: 36.12
Profit: 13.9 - max potential: 36.5 
Average growth rate: 9.441, consistent: 1, project growth: 6.147
DCF: 31046.4, market_dcf: 34.3, adjusted_dcf: 4726.6, dcf_to_price : 17756.44



**********- Score: 12.199999809265137 - IAC - NasdaqGS: 56.62 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 74.32 - min: 47.24 - max: 76.9
Profit: 31.3 - max potential: 35.8 
Average growth rate: 4.88, consistent: 1, project growth: 3.364
DCF: -4744914.5, market_


**********- Score: -27.600000381469727 - AXAS - NasdaqCM: 0.72 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 1.92 - min: 0.72 - max: 2.42
Profit: 166.7 - max potential: 236.1 
Average growth rate: 40.522, consistent: 1, project growth: 26.532
DCF: -13863489007.6, market_dcf: -176.2, adjusted_dcf: -1776859754.3, dcf_to_price : -246786077086.11



**********- Score: -0.699999988079071 - LCUT - NasdaqGS: 3.54 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 7.15 - min: 0.99 - max: 7.1936
Profit: 102.0 - max potential: 103.2 
Average growth rate: 10.252, consistent: 1, project growth: 6.903
DCF: -69153.5, market_dcf: -6.7, adjusted_dcf: -12521.2, dcf_to_price : -353806.21



**********- Score: 2.5999999046325684 - PGNT - OTCPK: 2.78 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 2.35 - min: 2.01 - max: 3.15
Profit: -15.5 - max potential: 13.3 
Average growth rate: 3.28, consistent: 1, project growth: 2.121
DCF

In [33]:
print("""
Initial Investment: ${}
Average yearly gain : {},
Average potential gain: {}
Cumulated potential capital: ${}
Cumulated capital yearly end: ${}

""".format(
    initial_investment,
    round(numpy.mean(profit_yearly_gains)),
    round(numpy.mean(potential_yearly_gains)),
    '{:0,.2f}'.format(yearly_cumulated_gain),
    '{:0,.2f}'.format(potential_gain),
    
))


Initial Investment: $1000
Average yearly gain : 36.0,
Average potential gain: 79.0
Cumulated potential capital: $2,533.35
Cumulated capital yearly end: $12,509.68




# Shorting - current year 2019

In [10]:
report_short_all(2019)


**********- Score: -41.20000076293945 - HMN - NYSE: 43.66 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 36.15 - min: 31.3 - max: 45.66
Profit: 17.2 - max potential: 28.3 
Average growth rate: -1.08, consistent: 1, project growth: -1.414
DCF: -13199.9, market_dcf: -525.2, adjusted_dcf: -27834.8, dcf_to_price : -63853.55



**********- Score: 8.899999618530273 - RYI - NYSE: 11.83 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 5.96 - min: 3.94 - max: 12.43
Profit: 49.6 - max potential: 66.7 
Average growth rate: -0.341, consistent: 1, project growth: -0.453
DCF: 118.4, market_dcf: 56.3, adjusted_dcf: 170.8, dcf_to_price : 1343.79



**********- Score: 28.0 - X - NYSE: 11.41 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 8.15 - min: 4.9 - max: 11.41
Profit: 28.6 - max potential: 57.1 
Average growth rate: -0.341, consistent: 1, project growth: -0.445
DCF: -58.7, market_dcf: 28.6, adjusted_dcf: -87.4, dcf_to_

(18.436363636363634, 47.94545454545455)

In [18]:
report_short_all(2018)


**********- Score: 6.800000190734863 - TG - NYSE: 15.86 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 22.35 - min: 15.59 - max: 23.31
Profit: -40.9 - max potential: 1.7 
Average growth rate: -4.675, consistent: 1, project growth: -6.307
DCF: 0.0, market_dcf: 22.3, adjusted_dcf: 0.0, dcf_to_price : -100.0



**********- Score: -4.599999904632568 - LNC - NYSE: 51.31 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 59.01 - min: 50.84 - max: 67.15
Profit: -15.0 - max potential: 0.9 
Average growth rate: -4.335, consistent: 1, project growth: -5.718
DCF: 0.0, market_dcf: 8227.2, adjusted_dcf: 0.0, dcf_to_price : -100.0



**********- Score: 2.5 - UAL - NasdaqGS: 83.73 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 88.09 - min: 77.49 - max: 95.28
Profit: -5.2 - max potential: 7.5 
Average growth rate: -3.985, consistent: 1, project growth: -5.296
DCF: 0.1, market_dcf: 157.9, adjusted_dcf: 0.1, dcf_to_price : -99

(-16.24, 18.424)

In [19]:
!ipython nbconvert --to html Reporting_Rank2_2020_06_16_consistent_UP_DOWN.ipynb

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook Reporting_Rank2_2020_06_16_consistent_UP_DOWN.ipynb to html
[NbConvertApp] Writing 452789 bytes to Reporting_Rank2_2020_06_16_consistent_UP_DOWN.html
